# Introduction to the Reverse Osmosis Unit Model
This tutorial shows how to build, initialize, simulate, and optimize a reverse osmosis (RO) unit model using WaterTAP. In this example, we will minimize the specific energy consumption of the RO unit.

<p align="center">
<img src='img/wf_8steps.svg' alt="Basic Worflow" width="500">
</p>

## Step 1: Import libraries from Pyomo, IDAES and WaterTAP.

<p align="center">
<img src='img/wf_8steps_1.svg' alt="Basic Worflow" width="500">
</p>

In [1]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Objective,
    Expression,
    value,
)  # including "value" for getting the value of Pyomo objects

# Imports from IDAES
from idaes.core import FlowsheetBlock  # Import flowsheet block from IDAES core
from watertap.core.solvers import get_solver  # Import function to get default solver
from idaes.core.util.model_statistics import (
    degrees_of_freedom,
)  # Import function to check degrees of freedom
from idaes.core.util.scaling import (
    calculate_scaling_factors,
    set_scaling_factor,
)  # Import utility function for calculating scaling factors


# Imports from WaterTAP
from watertap.property_models.NaCl_prop_pack import (
    NaClParameterBlock,
)  # Import NaCl property model
from watertap.unit_models.reverse_osmosis_0D import ReverseOsmosis0D  # Import RO model
from watertap.core import ConcentrationPolarizationType, MassTransferCoefficient

## Step 2: Build the RO model.

<p align="center">
<img src='img/wf_8steps_2.svg' alt="Basic Worflow" width="500">
</p>

Create a Pyomo concrete model, flowsheet, and NaCl property parameter block. Add an RO unit to the flowsheet.



In [2]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.unit = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type=ConcentrationPolarizationType.none,
    mass_transfer_coefficient=MassTransferCoefficient.none,
    has_pressure_change=False,
)

## Step 3: Specify values for system variables.

<p align="center">
<img src='img/wf_8steps_3.svg' alt="Basic Worflow" width="500">
</p>

In [3]:
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "NaCl"].fix(
    0.035
)  # mass flow rate of NaCl (kg/s)
m.fs.unit.inlet.flow_mass_phase_comp[0, "Liq", "H2O"].fix(
    0.965
)  # mass flow rate of water (kg/s)
m.fs.unit.inlet.pressure[0].fix(50e5)  # feed pressure (Pa)
m.fs.unit.inlet.temperature[0].fix(298.15)  # feed temperature (K)
m.fs.unit.area.fix(50)  # membrane area (m^2)
m.fs.unit.A_comp.fix(4.2e-12)  # membrane water permeability (m/Pa/s)
m.fs.unit.B_comp.fix(3.5e-8)  # membrane salt permeability (m/s)
m.fs.unit.permeate.pressure[0].fix(101325)  # permeate pressure (Pa)

## Step 4: Scale all variables.

<p align="center">
<img src='img/wf_8steps_4.svg' alt="Basic Worflow" width="500">
</p>

Set scaling factors for component mass flowrates, membrane area and all other variables.


In [4]:
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1, index=("Liq", "H2O"))
m.fs.properties.set_default_scaling("flow_mass_phase_comp", 1e2, index=("Liq", "NaCl"))
set_scaling_factor(m.fs.unit.area, 1e-2)
calculate_scaling_factors(m)

## Step 5: Initialize the model.

<p align="center">
<img src='img/wf_8steps_5.svg' alt="Basic Worflow" width="500">
</p>

In [5]:
m.fs.unit.initialize()

2026-01-15 04:58:23 [INFO] idaes.init.fs.unit.feed_side: Initialization Complete
2026-01-15 04:58:23 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found


## Step 6: Setup a solver and run a simulation.
Check that degrees of freedom = 0 before attempting simulation. This means that the performance of the flowsheet is completely determined by the system variables that were fixed above. Then run the simulation and report the results.

<p align="center">
<img src='img/wf_8steps_6.svg' alt="Basic Worflow" width="500">
</p>

In [6]:
assert degrees_of_freedom(m) == 0  # Check DOF
solver = get_solver()  # Setup solver
simulation_results = solver.solve(m)  # Run simulation
m.fs.unit.report()  # Display report, reports include a small subset of the most important variables


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  50.000 :    meter ** 2 :  True : (0.1, 100000.0)
    Solvent Mass Recovery Rate : 0.30933 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.30646 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')   kilogram / second     0.96500     0.66649        0.29851   
    flow_mass_phase_comp ('Liq', 'NaCl')  kilogram / second    0.035000    0.034924     7.5676e-05   
    temperature                                      kelvin 

In [7]:
assert degrees_of_freedom(m) == 0  # Check DOF
solver = get_solver()  # Setup solver
simulation_results = solver.solve(m)  # Run simulation
m.fs.unit.report()  # Display report, reports include a small subset of the most important variables


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  50.000 :    meter ** 2 :  True : (0.1, 100000.0)
    Solvent Mass Recovery Rate : 0.30933 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.30646 : dimensionless : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')   kilogram / second     0.96500     0.66649        0.29851   
    flow_mass_phase_comp ('Liq', 'NaCl')  kilogram / second    0.035000    0.034924     7.5676e-05   
    temperature                                      kelvin 

In [8]:
# Display all results, this shows all variables and constraints
m.fs.unit.display()

Block fs.unit

  Variables:
    area : Total Membrane area
        Size=1, Index=None, Units=m**2
        Key  : Lower : Value : Upper    : Fixed : Stale : Domain
        None :   0.1 :    50 : 100000.0 :  True :  True : NonNegativeReals
    _flow_mass_phase_comp_inlet_ref : Size=2, Index=fs._time*fs.properties.phase_list*fs.properties.component_list, ReferenceTo=fs.unit.feed_side.properties_in[...].component('flow_mass_phase_comp')[...]
        Key                  : Lower : Value : Upper : Fixed : Stale : Domain
         (0.0, 'Liq', 'H2O') :   0.0 : 0.965 :  None :  True :  True : NonNegativeReals
        (0.0, 'Liq', 'NaCl') :   0.0 : 0.035 :  None :  True :  True : NonNegativeReals
    _temperature_inlet_ref : Size=1, Index=fs._time, ReferenceTo=fs.unit.feed_side.properties_in[...].component('temperature')
        Key : Lower  : Value  : Upper  : Fixed : Stale : Domain
        0.0 : 273.15 : 298.15 : 373.15 :  True :  True : NonNegativeReals
    _pressure_inlet_ref : Size=1, Index

## Step 7: Optimize RO unit

<p align="center">
<img src='img/wf_8steps_7.svg' alt="Basic Worflow" width="500">
</p>

### Step 7.1: Unfix variables (membrane area, feed pressure) and set variable bounds

<p align="center">
<img src='img/wf_8steps_7-1.svg' alt="Basic Worflow" width="500">
</p>

In [9]:
m.fs.unit.area.unfix()  # unfix membrane area (m^2)
m.fs.unit.inlet.pressure[0].unfix()  # unfix feed pressure (Pa)

# Set lower and upper bounds for membrane area (m^2)
m.fs.unit.area.setlb(1)
m.fs.unit.area.setub(500)

# Set lower and upper bounds for feed pressure (Pa)
m.fs.unit.inlet.pressure[0].setlb(10e5)
m.fs.unit.inlet.pressure[0].setub(80e5)

## Step 7.2: Add constraints and objective 

<p align="center">
<img src='img/wf_8steps_7-2.svg' alt="Basic Worflow" width="500">
</p>

Minimize the specific energy consumption (energy / permeate flow) with a 50% water recovery constraint.

In [10]:
# Assume 100% efficiency of pumps and ERD and no pressure losses
# --> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0] / (3.6e6)
)

# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0, "Liq"].fix(0.50)

### Step 7.3: Run optimization to minimize specific energy consumption

<p align="center">
<img src='img/wf_8steps_7-3.svg' alt="Basic Worflow" width="500">
</p>

The optimization problem will find the membrane area and inlet pressure that achieve 50% recovery while minimizing specific energy consumption. 

In [11]:
# Check DOF
print(degrees_of_freedom(m))

# Solve the model
optimization_results = solver.solve(m)

1


## Step 8: Analyze Results

<p align="center">
<img src='img/wf_8steps_8.svg' alt="Basic Worflow" width="500">
</p>

<div class="alert alert-block alert-info">
<b>Note:</b> You can analyze and visualize model status and results at any point during the process, not strictly at the end.
</div>

In [12]:
print(optimization_results)



Problem: 
- Lower bound: -.inf
  Upper bound: .inf
  Number of objectives: 1
  Number of constraints: 92
  Number of variables: 93
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04057574272155762
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [13]:

# Display results of the optimized system
print(f"Optimized membrane area: {value(m.fs.unit.area)}")
print(f"Optimized membrane pressure: {value(m.fs.unit.inlet.pressure[0])}")
print(f"Specific energy consumption: {value(m.fs.specific_energy_consumption)}")

Optimized membrane area: 143.93509255062406
Optimized membrane pressure: 4526950.048349283
Specific energy consumption: 1.2574861245414672


In [14]:

m.fs.unit.report()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value   : Units         : Fixed : Bounds
                 Membrane Area :  143.94 :    meter ** 2 : False : (1, 500)
    Solvent Mass Recovery Rate : 0.50466 : dimensionless : False : (0.01, 0.999999)
      Volumetric Recovery Rate : 0.50000 : dimensionless :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Units         Feed Inlet  Feed Outlet  Permeate Outlet
    flow_mass_phase_comp ('Liq', 'H2O')   kilogram / second     0.96500     0.47801        0.48699   
    flow_mass_phase_comp ('Liq', 'NaCl')  kilogram / second    0.035000    0.034778     0.00022231   
    temperature                                      kelvin      29